# Task: Edge classification with pytorch geometric
### Chenguang Guan

In [1]:
import torch
import torch_geometric
from torch import Tensor

import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing
from torch.nn import Sequential as Seq, Linear, ReLU, Sigmoid

import random
from torch.utils.data import Dataset
from torch_geometric.loader import DataLoader
from pathlib import Path

## A. Dataset Preparation
We can either directly load the dataset and pass it to a list, or define a custom Dataset class as the tutorial.

### A-I. The Naive Way
The advantage point is that we can easily seperate the training set and test set.

In [2]:
def naive_dataset(path: Path):
    total_graphs = list()
    for i in range(10):
        subdir = "batch_1_" + str(i) + "/"
        total_graphs += list(path.glob(subdir + "*.pt"))
    dataset = list()
    for j in range(len(total_graphs)):
        dataset.append(torch.load(total_graphs[j]))
    return dataset
dataset = naive_dataset(Path("./data_gsoc"))

In [3]:
random.shuffle(dataset)
train_loader = DataLoader(dataset[:int(9/10*len(dataset))], batch_size=32)
test_loader = DataLoader(dataset[int(9/10*len(dataset)):], batch_size=32)

### A-II. Custom Dataset

In [4]:
class MyDataset(Dataset):
    def __init__(self, path: Path):
        super().__init__()
        total_graphs = list()
        for i in range(10):
            subdir = "batch_1_" + str(i) + "/"
            total_graphs += list(path.glob(subdir + "*.pt"))
        
        #random.shuffle(total_graphs)
        self.graphs = total_graphs
    
    def __getitem__(self, idx):
        return torch.load(self.graphs[idx])
    
    def __len__(self) -> int:
        return len(self.graphs)
    
    def shuffle(self):
        random.shuffle(self.graphs)
        
#dataset = MyDataset(Path("./data_gsoc"))
#dataset.shuffle()

### A-III. Attributes of The Graph Data

In [5]:
print(dataset[0])
print("num of nodes:",dataset[0].num_nodes)
print("num of edges:",dataset[0].num_edges)
print("num of node features:",dataset[0].num_node_features)
print("num of edge features:",dataset[0].num_edge_features)
print("Whether it's a directed graph?",dataset[0].is_directed())

Data(x=[408, 6], edge_index=[2, 3554], edge_attr=[3554, 4], y=[3554])
num of nodes: 408
num of edges: 3554
num of node features: 6
num of edge features: 4
Whether it's a directed graph? True


This is a directed graph with 6-d node features and 4-d edge features, and edge index is written in the Coordinate format (COO):
$$ G = (X, R, I) $$
Where,
$$ X = R^{n_{nodes}\times 6} $$
$$ R = R^{n_{edges}\times 4} $$
$$ I = N^{2\times n_{edges}} $$
$I[0,i]$ is the source node of the i-th edge, and $I[1,i]$ is the target node of the i-th edge.
While we also have traning target $Y=\{0,1\}^{n_{edges}}$, which is tha ground truth of the edge classification.

## B. Models

The main idea of graph neural network is to update the node embedding vectors and edge embedding vectors based on locality and message passing.
The general form of the updating is:
$$x_i^{(t)}=\phi_{\text {node }}^{(t)}\left(x_i^{(t-1)}, \underset{j \in N(i)}{\square} \phi_{\text {message }}^{(t)}\left(x_i^{(t-1)}, x_j^{(t-1)}, a_{i j}^{(t-1)}\right)\right)$$


### B-I. Interaction Networks
This model comes from "DeZoort, G., Thais, S., Duarte, J. et al. Charged Particle Tracking via Edge-Classifying Interaction Networks. Comput Softw Big Sci 5, 26 (2021). https://doi.org/10.1007/s41781-021-00073-z"

In this model:
1. We take the "aggregation" as "add": $\underset{j \in N(i)}{\square} = \sum_{j \in N(i)} $.
2. We assign $\phi_{\text {message }}^{(t)}\left(x_i^{(t-1)}, x_j^{(t-1)}, e_{i j}^{(t-1)}\right) $ to $a_{ij}^{(t)}$, which means that $a_{ij}^{(t)}= \phi_{\text {message }}^{(t)}\left(x_i^{(t-1)}, x_j^{(t-1)}, a_{i j}^{(t-1)}\right)$ and $x_i^{(t)}=\phi_{\text {node }}^{(t)}\left(x_i^{(t-1)},\underset{j \in N(i)}{\square}a_{i j}^{(t-1)}\right) $.

In the literature:
1. The authors take $\phi_{\text {message }} \rightarrow \phi_{R, 1}$ and $\phi_{\text {node }} \rightarrow \phi_O$ as MLP.
2. The authors take only one time step (only update the embedding vectors one time)
$$a_{i j}^{(1)}=\phi_{R, 1}\left(x_i^{(0)}, x_j^{(0)}, a_{i j}^{(0)}\right)$$
$$x_i^{(1)}=\phi_O\left(x_i^{(0)}, \sum_{j \in N(i)} a_{i j}^{(1)}\right)$$

3. We also need an extra layer to transform the embedding vectors to classification results (also called weights):
$$w_{i j}^{(1)}:=\phi_{R, 2}\left(x_i^{(1)}, x_j^{(1)}, a_{i j}^{(1)}\right)$$

4. $\phi_{R, 1}$ and $\phi_{R, 2}$ are called Relational Models, and $\phi_{O}$ is called Object Model.

In [6]:
class MLP_Layer(nn.Module):
    def __init__(self, input_size, output_size, hidden_size):
        super(MLP_Layer, self).__init__()

        self.layers = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size),
        )

    def forward(self, m):
        return self.layers(m)

In [7]:
class InteractionNetwork(MessagePassing):
    def __init__(self, hidden_size):
        super(InteractionNetwork, self).__init__(aggr='add', 
                                                 flow='source_to_target')
        self.R1 = MLP_Layer(16, 4, hidden_size)
        self.O = MLP_Layer(10, 6, hidden_size)
        self.R2 = MLP_Layer(16, 1, hidden_size)
        self.E: Tensor = Tensor()

    def forward(self, x: Tensor, edge_index: Tensor, edge_attr: Tensor) -> Tensor:

        # propagate_type: (x: Tensor, edge_attr: Tensor)
        x_tilde = self.propagate(edge_index, x=x, edge_attr=edge_attr, size=None)
        m2 = torch.cat([x_tilde[edge_index[1]],
                        x_tilde[edge_index[0]],
                        self.E], dim=1)
        return torch.sigmoid(self.R2(m2))

    def message(self, x_i, x_j, edge_attr):
        # x_i --> incoming
        # x_j --> outgoing        
        m1 = torch.cat([x_i, x_j, edge_attr], dim=1)
        self.E = self.R1(m1)
        return self.E

    def update(self, aggr_out, x):
        c = torch.cat([x, aggr_out], dim=1)
        return self.O(c) 

### B-II. Multi-Layer Interaction Network
We can also stack more layers of InteractionNetwork before moving to the edge weighting stage ($w_{i j}^{(1)}=\phi_{R, 2}\left(x_i^{(1)}, x_j^{(1)}, a_{i j}^{(1)}\right) $):

In [8]:
class InteractionNetwork_wo_weight(MessagePassing):
    def __init__(self, hidden_size):
        super(InteractionNetwork_wo_weight, self).__init__(aggr='add', 
                                                 flow='source_to_target')
        self.R1 = MLP_Layer(16, 4, hidden_size)
        self.O = MLP_Layer(10, 6, hidden_size)
        #self.R2 = MLP_Layer(16, 1, hidden_size)
        self.E: Tensor = Tensor()

    def forward(self, x: Tensor, edge_index: Tensor, edge_attr: Tensor) -> Tensor:

        # propagate_type: (x: Tensor, edge_attr: Tensor)
        x_tilde = self.propagate(edge_index, x=x, edge_attr=edge_attr, size=None)
        return x_tilde, self.E

    def message(self, x_i, x_j, edge_attr):
        # x_i --> incoming
        # x_j --> outgoing        
        m1 = torch.cat([x_i, x_j, edge_attr], dim=1)
        self.E = self.R1(m1)
        return self.E

    def update(self, aggr_out, x):
        c = torch.cat([x, aggr_out], dim=1)
        return self.O(c) 

In [9]:
class Multi_IN(nn.Module):
    def __init__(self, hidden_size):
        super(Multi_IN, self).__init__()
        
        #self.IN_layer = nn.Sequential(InteractionNetwork_wo_weight(hidden_size), InteractionNetwork_wo_weight(hidden_size))
        self.R2 = MLP_Layer(16, 1, hidden_size)

    def forward(self, x: Tensor, edge_index: Tensor, edge_attr: Tensor, num_layers = 1) -> Tensor:
        for i in range(num_layers):
            x, edge_attr = InteractionNetwork_wo_weight(hidden_size)(x=x,edge_index=edge_index,edge_attr=edge_attr)
        m2 = torch.cat([x[edge_index[1]], x[edge_index[0]], edge_attr], dim=1)
        return torch.sigmoid(self.R2(m2))

### B-III. DIY Models
A lot of models do not consider edge features in their updating schemes, such as famous GCN (https://arxiv.org/abs/1609.02907) and Edge Convolution (https://arxiv.org/abs/1801.07829). Therefore, one naive idea is to directly concatenate the edge embedding vectors with node embedding vectors when doing message passing.

#### The Variant of Edge Convolution (DIY)
The orginal Edge Convolution is defined as:
$$x_i^{(k)}=\max _{j \in N(i)} h_{\Theta}\left(x_i^{(k-1)}, x_j^{(k-1)}-x_i^{(k-1)}\right)$$

Here, $h_{\Theta}$ is also a MLP. The original Edge Convolution is suitable for the network without edge features.

Therefore, we can modify the Edge Convolution and include node features into the message passing.
$$a_{ij}^{(k)}= h_{\Theta}\left(x_i^{(k-1)}, x_j^{(k-1)}-x_i^{(k-1)},a_{ij}^{(k-1)}\right)$$
$$x_i^{(1)}=\phi_O\left(x_i^{(k)}, \max _{j \in N(i)} a_{i j}^{(k-1)}\right)$$

We also have two choices for the weights calculation:
1. Same as Interacting Network:$$w_{i j}^{(k)}:=\phi_{R, 2}\left(x_i^{(k)}, x_j^{(k)}, a_{i j}^{(k)}\right)$$
2. Or Similar with Edge Convolution Layer: $$w_{i j}^{(k)}:=\phi_{R, 2}\left(x_i^{(k)}, x_j^{(k)} - x_i^{(k)}, a_{i j}^{(k)}\right)$$

Interacting network in the literature and our DIY Edge Convolution can be both included in the general framework of graph neural network. 
$$x_i^{(t)}=\phi_{\text {node }}^{(t)}\left(x_i^{(t-1)}, \underset{j \in N(i)}{\square} \phi_{\text {message }}^{(t)}\left(x_i^{(t-1)}, x_j^{(t-1)}, a_{i j}^{(t-1)}\right)\right)$$

The differences between them are:
1. The form of $\phi_{\text {node }}$ is different.
2. $\underset{j \in N(i)}{\square}$ in Interacting Network is sum, while $\underset{j \in N(i)}{\square}$ in DIY edge convolution is max.

#### Type-I
$$w_{i j}^{(k)}:=\phi_{R, 2}\left(x_i^{(k)}, x_j^{(k)}, a_{i j}^{(k)}\right)$$

In [10]:
class EdgeConv_type1(MessagePassing):
    def __init__(self,hidden_size):
        super().__init__(aggr='max') #  "Max" aggregation.
        self.mlp = Seq(Linear(16, 4),
                       ReLU(),
                       Linear(4, 4))
        self.mlp_2 = Seq(Linear(10, 6),
                       ReLU(),
                       Linear(6, 6))
        self.R2 = nn.Sequential(
            nn.Linear(16, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 1),
        )
        self.E: Tensor = Tensor()

    def forward(self, x, edge_index, edge_attr):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]

        x_tilde = self.propagate(edge_index, x=x, edge_attr=edge_attr, size=None)
        m2 = torch.cat([x_tilde[edge_index[1]],
                        x_tilde[edge_index[0]]-x_tilde[edge_index[1]],
                        self.E], dim=1)
        return torch.sigmoid(self.R2(m2))

    def message(self, x_i, x_j, edge_attr):
        # x_i has shape [E, in_channels]
        # x_j has shape [E, in_channels]

        tmp = torch.cat([x_i, x_j, edge_attr], dim=1)  # tmp has shape [E, in_channels]
        self.E =self.mlp(tmp)
        
        return self.E
    
    def update(self, aggr_out, x):
        c = torch.cat([x, aggr_out], dim=1)
        return self.mlp_2(c) 

#### Type-II
$$w_{i j}^{(k)}:=\phi_{R, 2}\left(x_i^{(k)}, x_j^{(k)} - x_i^{(k)}, a_{i j}^{(k)}\right)$$

In [11]:
class EdgeConv_type2(MessagePassing):
    def __init__(self, hidden_size):
        super().__init__(aggr='max') #  "Max" aggregation.
        self.mlp = Seq(Linear(16, 4),
                       ReLU(),
                       Linear(4, 4))
        self.mlp_2 = Seq(Linear(10, 6),
                       ReLU(),
                       Linear(6, 6))
        self.R2 = nn.Sequential(
            nn.Linear(16, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 1),
        )
        self.E: Tensor = Tensor()

    def forward(self, x, edge_index, edge_attr):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]

        x_tilde = self.propagate(edge_index, x=x, edge_attr=edge_attr, size=None)
        m2 = torch.cat([x_tilde[edge_index[1]],
                        x_tilde[edge_index[0]]-x_tilde[edge_index[1]],
                        self.E], dim=1)
        return torch.sigmoid(self.R2(m2))

    def message(self, x_i, x_j, edge_attr):
        # x_i has shape [E, in_channels]
        # x_j has shape [E, in_channels]

        tmp = torch.cat([x_i, x_j - x_i, edge_attr], dim=1)  # tmp has shape [E, in_channels]
        self.E =self.mlp(tmp)
        
        return self.E
    
    def update(self, aggr_out, x):
        c = torch.cat([x, aggr_out], dim=1)
        return self.mlp_2(c) 

## C. Training and Results

Before moving to explicit models, we can define the general train loop function and test loop function. Here, we take the binary cross entropy as loss function.

In [12]:
loss_fn = F.binary_cross_entropy

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, X in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X.x,X.edge_index,X.edge_attr)
        #print(pred.shape)
        #print(X.y.shape)
        loss = loss_fn(pred.squeeze(), X.y, reduction='mean')

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    # size = len(dataloader.dataset)
    # size_1 = len(dataloader.dataset.Y)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    size_2 = 0
    with torch.no_grad():
        for X in dataloader:
            pred = model(X.x,X.edge_index,X.edge_attr)
            test_loss += loss_fn(pred.squeeze(), X.y, reduction='mean').item()
            correct += (pred.argmax(1) == X.y).type(torch.float).sum().item()
            size_2 += X.y.size(0)
    test_loss /= num_batches
    correct /= size_2
    #print(size,size_2)
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [14]:
import time

### C-1. Training and Evaluation of single layer IN:

In [9]:
learning_rate = 1e-3
hidden_size = 16
device = "cpu"
model_1 = InteractionNetwork(hidden_size).to(device)

In [18]:
optimizer_1 = torch.optim.SGD(model_1.parameters(), lr=learning_rate)
start_time = time.time()
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_loader, model_1, loss_fn, optimizer_1)
    test_loop(test_loader, model_1, loss_fn)
print("Done!")
end_time = time.time()
print("Consumed Time: ",end_time-start_time)

Epoch 1
-------------------------------
loss: 0.537301  [   32/ 8096]
loss: 0.510489  [ 3232/ 8096]
loss: 0.456089  [ 6432/ 8096]
Test Error: 
 Accuracy: 82.3%, Avg loss: 0.424709 

Epoch 2
-------------------------------
loss: 0.394871  [   32/ 8096]
loss: 0.419034  [ 3232/ 8096]
loss: 0.406836  [ 6432/ 8096]
Test Error: 
 Accuracy: 82.3%, Avg loss: 0.390649 

Epoch 3
-------------------------------
loss: 0.358287  [   32/ 8096]
loss: 0.401716  [ 3232/ 8096]
loss: 0.397869  [ 6432/ 8096]
Test Error: 
 Accuracy: 82.3%, Avg loss: 0.383816 

Epoch 4
-------------------------------
loss: 0.351883  [   32/ 8096]
loss: 0.396468  [ 3232/ 8096]
loss: 0.393665  [ 6432/ 8096]
Test Error: 
 Accuracy: 82.3%, Avg loss: 0.380231 

Epoch 5
-------------------------------
loss: 0.348766  [   32/ 8096]
loss: 0.393224  [ 3232/ 8096]
loss: 0.390836  [ 6432/ 8096]
Test Error: 
 Accuracy: 82.3%, Avg loss: 0.377672 

Done!


### C-2. Training and Evaluation of multi layer IN:

we need to modify our train loop function and test loop function (introducing num of layers) for multi layer IN.

In [14]:
def train_loop_2(dataloader, model, loss_fn, optimizer, num_layers):
    size = len(dataloader.dataset)
    for batch, X in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X.x,X.edge_index,X.edge_attr,num_layers)
        #print(pred.shape)
        #print(X.y.shape)
        loss = loss_fn(pred.squeeze(), X.y, reduction='mean')

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop_2(dataloader, model, loss_fn, num_layers):
    #size = len(dataloader.dataset)
    size_2 = 0
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X in dataloader:
            pred = model(X.x,X.edge_index,X.edge_attr, num_layers)
            test_loss += loss_fn(pred.squeeze(), X.y, reduction='mean').item()
            correct += (pred.argmax(1) == X.y).type(torch.float).sum().item()
            size_2 += X.y.size(0)
    test_loss /= num_batches
    correct /= size_2
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [15]:
learning_rate = 1e-3
hidden_size = 16
device = "cpu"
num_layers = 2

In [16]:
model_2 = Multi_IN(hidden_size).to(device)
optimizer_2 = torch.optim.SGD(model_2.parameters(), lr=learning_rate)
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop_2(train_loader, model_2, loss_fn, optimizer_2, num_layers)
    test_loop_2(test_loader, model_2, loss_fn, num_layers)
print("Done!")

Epoch 1
-------------------------------
loss: 0.710915  [   32/ 8096]
loss: 0.706690  [ 3232/ 8096]
loss: 0.668700  [ 6432/ 8096]
Test Error: 
 Accuracy: 82.4%, Avg loss: 0.671176 

Epoch 2
-------------------------------
loss: 0.677101  [   32/ 8096]
loss: 0.659391  [ 3232/ 8096]
loss: 0.648296  [ 6432/ 8096]
Test Error: 
 Accuracy: 82.4%, Avg loss: 0.638465 

Epoch 3
-------------------------------
loss: 0.633540  [   32/ 8096]
loss: 0.611605  [ 3232/ 8096]
loss: 0.637673  [ 6432/ 8096]
Test Error: 
 Accuracy: 82.4%, Avg loss: 0.608189 

Epoch 4
-------------------------------
loss: 0.614853  [   32/ 8096]
loss: 0.604393  [ 3232/ 8096]
loss: 0.598384  [ 6432/ 8096]
Test Error: 
 Accuracy: 82.4%, Avg loss: 0.590021 

Epoch 5
-------------------------------
loss: 0.594011  [   32/ 8096]
loss: 0.583257  [ 3232/ 8096]
loss: 0.570550  [ 6432/ 8096]
Test Error: 
 Accuracy: 82.4%, Avg loss: 0.568066 

Done!


### C-3. Training and Evaluation of DIY Edge Convolution (Type-I):

In [17]:
learning_rate = 1e-3
hidden_size = 16
device = "cpu"
model_3 = EdgeConv_type1(hidden_size).to(device)

In [18]:
optimizer_3 = torch.optim.SGD(model_3.parameters(), lr=learning_rate)
start_time = time.time()
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_loader, model_3, loss_fn, optimizer_3)
    test_loop(test_loader, model_3, loss_fn)
print("Done!")
end_time = time.time()
print("Consumed Time: ", end_time-start_time)

Epoch 1
-------------------------------
loss: 0.706257  [   32/ 8096]
loss: 0.686487  [ 3232/ 8096]
loss: 0.661853  [ 6432/ 8096]
Test Error: 
 Accuracy: 82.4%, Avg loss: 0.649256 

Epoch 2
-------------------------------
loss: 0.654482  [   32/ 8096]
loss: 0.632862  [ 3232/ 8096]
loss: 0.613839  [ 6432/ 8096]
Test Error: 
 Accuracy: 82.4%, Avg loss: 0.602310 

Epoch 3
-------------------------------
loss: 0.609763  [   32/ 8096]
loss: 0.588305  [ 3232/ 8096]
loss: 0.576335  [ 6432/ 8096]
Test Error: 
 Accuracy: 82.4%, Avg loss: 0.566171 

Epoch 4
-------------------------------
loss: 0.574952  [   32/ 8096]
loss: 0.554169  [ 3232/ 8096]
loss: 0.549161  [ 6432/ 8096]
Test Error: 
 Accuracy: 82.4%, Avg loss: 0.540704 

Epoch 5
-------------------------------
loss: 0.550318  [   32/ 8096]
loss: 0.529785  [ 3232/ 8096]
loss: 0.530514  [ 6432/ 8096]
Test Error: 
 Accuracy: 82.4%, Avg loss: 0.523548 

Done!
Consumed Time:  315.89234042167664


### C-4. Training and Evaluation of DIY Edge Convolution (Type-II):

In [19]:
learning_rate = 1e-3
hidden_size = 16
device = "cpu"
model_4 = EdgeConv_type2(hidden_size).to(device)

In [20]:
start_time = time.time()
optimizer_4 = torch.optim.SGD(model_4.parameters(), lr=learning_rate)
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_loader, model_4, loss_fn, optimizer_4)
    test_loop(test_loader, model_4, loss_fn)
print("Done!")
end_time = time.time()
print("Consumed Time: ", end_time-start_time)

Epoch 1
-------------------------------
loss: 0.714863  [   32/ 8096]
loss: 0.694697  [ 3232/ 8096]
loss: 0.681964  [ 6432/ 8096]
Test Error: 
 Accuracy: 82.4%, Avg loss: 0.673296 

Epoch 2
-------------------------------
loss: 0.674221  [   32/ 8096]
loss: 0.652884  [ 3232/ 8096]
loss: 0.646555  [ 6432/ 8096]
Test Error: 
 Accuracy: 82.4%, Avg loss: 0.638517 

Epoch 3
-------------------------------
loss: 0.640209  [   32/ 8096]
loss: 0.617357  [ 3232/ 8096]
loss: 0.616570  [ 6432/ 8096]
Test Error: 
 Accuracy: 82.4%, Avg loss: 0.608903 

Epoch 4
-------------------------------
loss: 0.611297  [   32/ 8096]
loss: 0.586711  [ 3232/ 8096]
loss: 0.590945  [ 6432/ 8096]
Test Error: 
 Accuracy: 82.4%, Avg loss: 0.583473 

Epoch 5
-------------------------------
loss: 0.586527  [   32/ 8096]
loss: 0.560110  [ 3232/ 8096]
loss: 0.569018  [ 6432/ 8096]
Test Error: 
 Accuracy: 82.4%, Avg loss: 0.561613 

Done!
Consumed Time:  299.5344567298889


## D. Further Directions
Due to the time limitation, I can not implement every ideas in the task. However, in the near future there are several directions I would like to contribute to:
### D-1. Network Structure
1. #### Next Nearest Neighbour:
The first way to improve the network I would like to try is to include next nearest neighbor for the message passing. We can extend the $\underset{j \in N(i)}{\square}$ to $\underset{j \in N(i)\cup N.N.(i)}{\square}$, where $N.N.(i)$ denotes next nearest neighbot of node-$i$.

2. #### Graph Attention Network 
We can also try some framework beyond Graph Convolutional Network Scheme. For example, one promising direction is using Graph Attention Network（GAT) in the task.

### D-2. Training Results (Bug: Fixed)
The accuracy on test dataset of four models are all apprxomately equal to $31\%$, which is very weird. There might be some bugs in the training or evaluation part of my code.
#### ! ----------I have fixed this bug--------------

### D-3. Evaluation
I would like to utilize ROC curve (Receiver Operating Characteristic curve) and AUC score (Area Under the ROC Curve) in the further evaluation.

### D-4. Class Imbalance Problem

In [54]:
num_0 = 0
num_1 = 0
for i in dataset[0].y:
    if i == 0:
        num_0 += 1
    elif i == 1:
        num_1 += 1
    else:
        print("error")
print("Num of class-0",num_0)
print("Num of class-1",num_1)

Num of class-0 882
Num of class-1 374


There is a significant class imbalance in our data. 

In the normal case, we can use under-sampling or over-sampling to fix the imbalanced class. However, in the graph problem this is tricky, because we can not arbitarily delete or add edges in a single graph.

Therefore, we might need to find a way to enlarge or reduce each graph (introducing extra nodes and edges with $y=1$ or properly deleting existing nodes and edges with $y=0$).

#### Ratio of two classes:

1. #### Training Set:
Class-0 : Class-1: 82.3 : 17.7
2. #### Test Set:
Class-0 : Class-1: 82.4 : 17.6

In [29]:
def Class_Imbalance(dataloader):
    # size = len(dataloader.dataset)
    # size_1 = len(dataloader.dataset.Y)
    num_batches = len(dataloader)
    size_2 = 0
    class_0 = 0
    class_1 = 0
    with torch.no_grad():
        for X in dataloader:
            class_0 += (X.y == 0).type(torch.float).sum().item()
            class_1 += (X.y == 1).type(torch.float).sum().item()
            size_2 += X.y.size(0)
    print(f"Number of Total Edges: {size_2}, Class-0: {(100*class_0/size_2):>0.1f}%, Class-1: {(100*class_1/size_2):>0.1f}% \n")

In [30]:
Class_Imbalance(train_loader)

Number of Total Edges: 31726730, Class-0: 82.3%, Class-1: 17.7% 



In [31]:
Class_Imbalance(test_loader)

Number of Total Edges: 3575094, Class-0: 82.4%, Class-1: 17.6% 



#### Trivial Results of Previous Training
We found that training results (82.3%-82.4%) of all four models suffer from class imbalance servely:
1. Single-layer IN: 82.3%
2. Multi-layer IN: 82.4%
3. Type-I DIY Edge Convolution: 82.4%
4. Type-II DIY Edge Convolution: 82.4%

#### "weight" in torch.nn.BCELoss
A naive way is to introduce "weight" (an argument of BCELoss function) in the Binary Eentropy Loss. The default setting of "weight" is None, which means that we treat two classes equaivalently.

## E. Trial Solution to Class Imbalance Problem:

Therefore, I can introduce the weight vector (such as $(0.177,0.823)$ ) in the loss function calculation.

### E-1. Weighted Cross Entropy Loss Function

In [8]:
#loss_fn_weighted = nn.CrossEntropyLoss(weight=torch.tensor([0.177, 0.823]) )
#loss_fn_weighted = nn.CrossEntropyLoss(weight=torch.tensor([0.05, 0.95]) )
#loss_fn_weighted = nn.CrossEntropyLoss(weight=torch.tensor([0.1, 0.9]) )
loss_fn_weighted = nn.CrossEntropyLoss(weight=torch.tensor([0.177, 0.823]) )

In [9]:
def train_loop_3(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, X in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X.x,X.edge_index,X.edge_attr)
        #print(pred.shape)
        #print(X.y.shape)
        #print(pred.shape)
        #print(X.y.shape)
        loss = loss_fn(pred, X.y.type(torch.LongTensor))

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop_3(dataloader, model, loss_fn):
    # size = len(dataloader.dataset)
    # size_1 = len(dataloader.dataset.Y)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    size_2 = 0
    with torch.no_grad():
        for X in dataloader:
            pred = model(X.x,X.edge_index,X.edge_attr)
            test_loss += loss_fn(pred, X.y.type(torch.LongTensor) ).item()
            correct += (pred.argmax(1) == X.y).type(torch.float).sum().item()
            size_2 += X.y.size(0)
    test_loss /= num_batches
    correct /= size_2
    #print(size,size_2)
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

### E-2. Output Layer of $\phi_{R,2}$:

I also revise the sigmoid and linear layer (final layer) of IN, which is the output layer of $\phi_{R,2}$:

The previous strcture is the mapping from $16-d$ vectors to $1-d$ scalar, while the current strctures is the mapping from $16-d$ vectors to $2-d$ vector (class-0 and class-1).

In [10]:
class InteractionNetwork_2(MessagePassing):
    def __init__(self, hidden_size):
        super(InteractionNetwork_2, self).__init__(aggr='add', 
                                                 flow='source_to_target')
        self.R1 = MLP_Layer(16, 4, hidden_size)
        self.O = MLP_Layer(10, 6, hidden_size)
        self.R2 = MLP_Layer(16, 2, hidden_size)
        self.E: Tensor = Tensor()

    def forward(self, x: Tensor, edge_index: Tensor, edge_attr: Tensor) -> Tensor:

        # propagate_type: (x: Tensor, edge_attr: Tensor)
        x_tilde = self.propagate(edge_index, x=x, edge_attr=edge_attr, size=None)
        m2 = torch.cat([x_tilde[edge_index[1]],
                        x_tilde[edge_index[0]],
                        self.E], dim=1)
        return torch.sigmoid(self.R2(m2))

    def message(self, x_i, x_j, edge_attr):
        # x_i --> incoming
        # x_j --> outgoing        
        m1 = torch.cat([x_i, x_j, edge_attr], dim=1)
        self.E = self.R1(m1)
        return self.E

    def update(self, aggr_out, x):
        c = torch.cat([x, aggr_out], dim=1)
        return self.O(c) 

### E-3. Results of Single-layer IN with Weighted Loss Function

In [11]:
learning_rate = 1e-1
hidden_size = 16
device = "cpu"
model_im = InteractionNetwork_2(hidden_size).to(device)

In [12]:
learning_rate_list = [1e-1] * 3 + [1e-2] * 2 + [1e-3] * 2 + [1e-4] * 2

In [15]:
epochs = len(learning_rate_list)
start_time = time.time()
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    optimizer_im = torch.optim.SGD(model_im.parameters(), lr=learning_rate_list[t])
    train_loop_3(train_loader, model_im, loss_fn_weighted, optimizer_im)
    test_loop_3(test_loader, model_im, loss_fn_weighted)
print("Done!")
end_time = time.time()
print("Consumed Time: ",end_time-start_time)

Epoch 1
-------------------------------
loss: 0.692971  [   32/ 8096]
loss: 0.619941  [ 3232/ 8096]
loss: 0.520998  [ 6432/ 8096]
Test Error: 
 Accuracy: 84.8%, Avg loss: 0.510816 

Epoch 2
-------------------------------
loss: 0.508774  [   32/ 8096]
loss: 0.506587  [ 3232/ 8096]
loss: 0.519499  [ 6432/ 8096]
Test Error: 
 Accuracy: 86.0%, Avg loss: 0.505861 

Epoch 3
-------------------------------
loss: 0.504040  [   32/ 8096]
loss: 0.503540  [ 3232/ 8096]
loss: 0.510444  [ 6432/ 8096]
Test Error: 
 Accuracy: 86.3%, Avg loss: 0.504886 

Epoch 4
-------------------------------
loss: 0.503002  [   32/ 8096]
loss: 0.502215  [ 3232/ 8096]
loss: 0.506346  [ 6432/ 8096]
Test Error: 
 Accuracy: 86.2%, Avg loss: 0.504744 

Epoch 5
-------------------------------
loss: 0.502965  [   32/ 8096]
loss: 0.502092  [ 3232/ 8096]
loss: 0.506292  [ 6432/ 8096]
Test Error: 
 Accuracy: 86.2%, Avg loss: 0.504653 

Epoch 6
-------------------------------
loss: 0.502859  [   32/ 8096]
loss: 0.502056  [ 32

With weighted loss function, the accuracy on test set is about $86.2\%-86.3\%$, which is better than $82.4\%$ (trivial case).

In [16]:
test_loop_3(train_loader, model_im, loss_fn_weighted)

Test Error: 
 Accuracy: 86.1%, Avg loss: 0.504318 



### E.4 Accuracy in Each Class

In [32]:
def Each_Class_Accuracy(dataloader, model):
    # size = len(dataloader.dataset)
    # size_1 = len(dataloader.dataset.Y)
    size_2 = 0
    class_0 = 0
    class_1 = 0
    class_0_correct = 0
    class_1_correct = 0
    with torch.no_grad():
        for X in dataloader:
            pred = model(X.x,X.edge_index,X.edge_attr)
            class_0 += (X.y == 0).type(torch.float).sum().item()
            class_1 += (X.y == 1).type(torch.float).sum().item()
            class_0_correct += torch.logical_and( (X.y == 0), (pred.argmax(1) == 0) ).type(torch.float).sum().item()
            class_1_correct += torch.logical_and( (X.y == 1), (pred.argmax(1) == 1) ).type(torch.float).sum().item()
            size_2 += X.y.size(0)
    print(f"Number of Total Edges: {size_2}, Class-0: {(100*class_0/size_2):>0.1f}%, Class-1: {(100*class_1/size_2):>0.1f}% \n")
    print(f"Class-0 Accuracy: {(100*class_0_correct/class_0):>0.1f}%, Class-1 Accuracy: {(100*class_1_correct/class_1):>0.1f}% \n")

In [33]:
Each_Class_Accuracy(test_loader, model_im)

Number of Total Edges: 3575094, Class-0: 82.4%, Class-1: 17.6% 

Class-0 Accuracy: 89.6%, Class-1 Accuracy: 70.4% 



We achieve $89.6\%$ for class-0 and $70.4\%$ for class-1, which is not a trivial result.